In [1]:
import networkx as nx
from networkx.readwrite import cytoscape_data
from rdkit import Chem
from rdkit.Chem import RDConfig
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
import cyjupyter
import os
from urllib import parse

In [2]:
mmp_path = os.path.join(RDConfig.RDContribDir, "mmpa/data/sample_mmps_default.csv")

In [3]:
lines = []
with open(mmp_path, 'r') as f:
    for line in f:
        lines.append(line.rstrip().split(','))


In [4]:
nodes = set()
for line in lines:
    nodes.add(line[0])
    nodes.add(line[1])
nodes = list(nodes)
print(str(len(nodes))+' mols')

10 mols


In [5]:
def smi2svg(smi):
    mol = Chem.MolFromSmiles(smi)
    try:
        Chem.rdmolops.Kekulize(mol)
    except:
        pass
    drawer = rdMolDraw2D.MolDraw2DSVG(400, 200)
    AllChem.Compute2DCoords(mol)
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText().replace("svg:", "")
    return svg

def smi2image(smi):
    svg_string = smi2svg(smi)
    impath = 'data:image/svg+xml;charset=utf-8,' + parse.quote(svg_string, safe="")
    return impath

In [6]:
g = nx.Graph()
for node in nodes:
    g.add_node(node, img=smi2image(node))
for line in lines:
    g.add_edge(line[0], line[1])

In [7]:
cy_g = cytoscape_data(g)

In [8]:
from cyjupyter import Cytoscape

In [9]:
stobj=[
  {'style': [{'css': {
      'background-color': 'blue',
      'shape' : 'rectangle',
      'width':800,
      'height':400,
      'border-color': 'rgb(0,0,0)',
      'border-opacity': 1.0,
      'border-width': 0.0,
      'color': '#4579e8',
      'background-image':'data(img)',
      'background-fit':'contain'
                    },
    'selector': 'node'},
            {'css': {
                'width': 20.0,
            },
            'selector': 'edge'}
            ],
  }]

In [10]:
cyobj=Cytoscape(data=cy_g, visual_style=stobj[0]['style'], layout_name='cose')

In [11]:
cyobj

Cytoscape(data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes': [{'data': {'img': 'd…

In [16]:
display(cyobj)

Cytoscape(data={'data': {}, 'elements': {'nodes': [{'data': {'img': 'data:image/svg+xml;charset=utf-8,%3C%3Fxm…